In [1]:
from copy import deepcopy

In [2]:
def parse_maze(mazestr):
    alphabet = 'ABCDEFGHIJKLMNOPQRSTUVWXYZ'
    maze = [[col.strip() for col in row.strip()] for row in mazestr.split('\n') if len(row) > 0]
    keys = {}
    doors = {}
    
    for i in range(len(maze)):
        for j in range(len(maze[0])):
            if maze[i][j] in alphabet:
                doors[maze[i][j]] = (i, j)
            elif maze[i][j] in alphabet.lower():
                keys[maze[i][j]] = (i, j)
            elif maze[i][j] == "@":
                origin = (i, j)
    
    return maze, origin, keys, doors

In [3]:
def manhattan(src, dst):
    dh = src[0] - dst[0]
    dw = src[1] - dst[1]
    return abs(dh) + abs(dw)

def astar(src, dst, maze, moves = [[0, 1], [0, -1], [1, 0], [-1, 0]]):
    blockers = '#ABCDEFGHIJKLMNOPQRSTUVWXYZ'
    openlist = [[manhattan(src, dst), 0, src[0], src[1]]]
    visitlist = []
    parents = {}
    
    done = False
    while len(openlist) > 0 and not done:
        visiting = openlist[0]
        visitlist.append((visiting[2], visiting[3]))
        openlist = openlist[1:]
        
        for move in moves:
            neighbour = (visiting[2] + move[0], visiting[3] + move[1])
            if neighbour[0] == dst[0] and neighbour[1] == dst[1]:
                parents[neighbour] = (visiting[2], visiting[3])
                done = True
                break
            
            if neighbour[0] < 0 or neighbour[1] < 0 or\
               neighbour[0] >= len(maze) or neighbour[1] >= len(maze[0]):
                continue
            
            if neighbour in visitlist:
                continue
            
            cell = maze[neighbour[0]][neighbour[1]]
            if cell in blockers or cell in blockers.lower():
                continue
                
            parents[neighbour] = (visiting[2], visiting[3])
            openlist.append([manhattan(neighbour, dst), visiting[1]+1, 
                             neighbour[0], neighbour[1]])
        openlist.sort()
            
    
    path = []
    if done:
        path.append(dst)
        parent = parents[dst]
        while not (parent == src):
            path.append(parent)
            parent = parents[parent]
        path.reverse()
        
    return path

In [4]:
def collect_keys(maze, origin, keys, doors, partial=[], distance=0, globest=1e9):
#     print('\nsearch map', partial)
#     for r in maze:
#         print(''.join([c for c in r]))

    if distance >= globest:
        return distance, partial

    if len(keys) == 0:
        if distance < globest:
            print(distance, partial)
            print('new global best!')
        return distance, partial
    
    mindist = len(maze)*len(maze[0])*26
    bestpart = []

    for key in keys:
        keypath = astar(origin, keys[key], maze)
        if len(keypath) > 0:
#             print('key {} at {} is reachable from {}'\
#                   .format(key, keys[key], origin))
            
            maze_i = deepcopy(maze)

            keys_i = deepcopy(keys)
            keyij = keys_i.pop(key)
            maze_i[keyij[0]][keyij[1]] = '@'
            maze_i[origin[0]][origin[1]] = '.'

            doors_i = deepcopy(doors)
            if key.upper() in doors:
                doorij = doors_i.pop(key.upper())
                maze_i[doorij[0]][doorij[1]] = '.'

            d, p = collect_keys(maze_i, keyij, keys_i, doors_i, 
                                partial.copy()+[key], distance+len(keypath), globest)

            if d < mindist:
                mindist = d
                bestpart = p
                
            if d < globest:
                globest = d
        
    return mindist, bestpart

In [5]:
mazestr="""\
#########
#b.A.@.a#
#########"""

maze, origin, keys, doors = parse_maze(mazestr)

print('maze under consideration')
for row in maze:
    print(''.join([c for c in row]))
    
print('\norigin', origin)
print('\nkey locations')
for key in keys:
    print(key, keys[key])
    
print('\ndoor locations')
for door in doors:
    print(door, doors[door])
    
print()
d, p = collect_keys(maze, origin, keys, doors)
print('\nbest:', d, p)

maze under consideration
#########
#b.A.@.a#
#########

origin (1, 5)

key locations
a (1, 7)
b (1, 1)

door locations
A (1, 3)

8 ['a', 'b']
new global best!

best: 8 ['a', 'b']


In [6]:
mazestr="""\
########################
#f.D.E.e.C.b.A.@.a.B.c.#
######################.#
#d.....................#
########################"""

maze, origin, keys, doors = parse_maze(mazestr)

print('maze under consideration')
for row in maze:
    print(''.join([c for c in row]))
    
print('\norigin', origin)
print('\nkey locations')
for key in keys:
    print(key, keys[key])
    
print('\ndoor locations')
for door in doors:
    print(door, doors[door])
    
print()
d, p = collect_keys(maze, origin, keys, doors)
print('\nbest:', d, p)

maze under consideration
########################
#f.D.E.e.C.b.A.@.a.B.c.#
######################.#
#d.....................#
########################

origin (1, 15)

key locations
e (1, 7)
f (1, 1)
c (1, 21)
a (1, 17)
d (3, 1)
b (1, 11)

door locations
E (1, 5)
B (1, 19)
A (1, 13)
D (1, 3)
C (1, 9)

114 ['a', 'b', 'c', 'e', 'd', 'f']
new global best!
86 ['a', 'b', 'c', 'd', 'e', 'f']
new global best!

best: 86 ['a', 'b', 'c', 'd', 'e', 'f']


In [7]:
mazestr="""\
########################
#...............b.C.D.f#
#.######################
#.....@.a.B.c.d.A.e.F.g#
########################"""

maze, origin, keys, doors = parse_maze(mazestr)

print(len(maze), len(maze[0]))

print('maze under consideration')
for row in maze:
    print(''.join([c for c in row]))
    
print('\norigin', origin)
print('\nkey locations')
for key in keys:
    print(key, keys[key])
    
print('\ndoor locations')
for door in doors:
    print(door, doors[door])
    
print()
d, p = collect_keys(maze, origin, keys, doors)
print('\nbest:', d, p)

5 24
maze under consideration
########################
#...............b.C.D.f#
#.######################
#.....@.a.B.c.d.A.e.F.g#
########################

origin (3, 6)

key locations
e (3, 18)
f (1, 22)
c (3, 12)
g (3, 22)
a (3, 8)
d (3, 14)
b (1, 16)

door locations
F (3, 20)
B (3, 10)
A (3, 16)
D (1, 20)
C (1, 18)

144 ['a', 'b', 'c', 'd', 'e', 'f', 'g']
new global best!
136 ['a', 'b', 'c', 'd', 'f', 'e', 'g']
new global best!
132 ['b', 'a', 'c', 'd', 'f', 'e', 'g']
new global best!

best: 132 ['b', 'a', 'c', 'd', 'f', 'e', 'g']


In [ ]:
mazestr="""\
#################
#i.G..c...e..H.p#
########.########
#j.A..b...f..D.o#
########@########
#k.E..a...g..B.n#
########.########
#l.F..d...h..C.m#
#################"""

maze, origin, keys, doors = parse_maze(mazestr)

print(len(maze), len(maze[0]))

print('maze under consideration')
for row in maze:
    print(''.join([c for c in row]))
    
print('\norigin', origin)cv
print('\nkey locations')
for key in keys:
    print(key, keys[key])
    
print('\ndoor locations')
for door in doors:
    print(door, doors[door])
    
print()
d, p = collect_keys(maze, origin, keys, doors)
print('\nbest:', d, p)

In [8]:
mazestr="""\
########################
#@..............ac.GI.b#
###d#e#f################
###A#B#C################
###g#h#i################
########################"""

maze, origin, keys, doors = parse_maze(mazestr)

print(len(maze), len(maze[0]))

print('maze under consideration')
for row in maze:
    print(''.join([c for c in row]))
    
print('\norigin', origin)
print('\nkey locations')
for key in keys:
    print(key, keys[key])
    
print('\ndoor locations')
for door in doors:
    print(door, doors[door])
    
print()
d, p = collect_keys(maze, origin, keys, doors)
print('\nbest:', d, p)

6 24
maze under consideration
########################
#@..............ac.GI.b#
###d#e#f################
###A#B#C################
###g#h#i################
########################

origin (1, 1)

key locations
e (2, 5)
h (4, 5)
f (2, 7)
c (1, 17)
i (4, 7)
g (4, 3)
a (1, 16)
d (2, 3)
b (1, 22)

door locations
I (1, 20)
G (1, 19)
C (3, 7)
A (3, 3)
B (3, 5)

85 ['e', 'f', 'a', 'c', 'i', 'd', 'g', 'b', 'h']
new global best!
83 ['e', 'a', 'c', 'f', 'i', 'd', 'g', 'b', 'h']
new global best!
81 ['a', 'c', 'f', 'i', 'd', 'g', 'b', 'e', 'h']
new global best!

best: 81 ['a', 'c', 'f', 'i', 'd', 'g', 'b', 'e', 'h']


In [ ]:
mazestr="""\
#################################################################################
#m....#...#...#.....#...#...............#.#...#.....#...........................#
#.###.#.#.#.#.###.#.#.#.#####.#.#########.#.#.#.#.###.###.#.#############.#####.#
#...#...#...#...#.#b#.#.#...#.#........x#.#.#.#.#.....#...#.#.......#.....#...#.#
#.#.###########.#.#.#.#.#.#.###########.#.#.#.#.#######.#####.#####.#######.#.###
#.#.......#...#.U.#...#...#...........#.#.#.#...#.....#..........p#.........#...#
#.#######.#.#.#######################.#.#.#.#####.###.###########.#############.#
#...#...#.#.#.#...#.S...#...........#...#.#.#...#.#.#.....#....k#.....#.......#.#
###.#.#.#.#.#.#.#.#.###.#.###.#####L###.#.#.#.#.#.#.###N###.###.#######.#####.#.#
#...#.#...#.#.#.#...#.#.#...#.#.......#.#.#...#...#.....#...#.#.#.......#...#.#.#
#.###.#####.#.#.#####.#.###.#.#########.#.#########.#####.###.#.#.#######.#.#.#.#
#.#.#.#...#.#.#.#.T...#.....#.........#.#...#.....#...#.#...#.....#.......#.#...#
#.#.#.#.#.#.#.#.#.###.#############.#.#.###.#.#.#####.#.###.#######.#####.#.#####
#.#.#q#.#.H.#.#.#.#...#.....F.#...#.#...#...#.#.....#.#...#...#.#...#.#...#...#.#
#.#.#.###.###.#.###.#.#####.#.#.#.###.###.###.###.#.#.###.###.#.#.###.#.#####.#.#
#...#...#.#...#.#...#.....#.#.#.#...#...#.#...#.#.#.#.#...#...#.......#.#.#...#.#
#######.#.#.###.#.#######.###.#.###.###.#.#.###.#.###.#.###.###########.#.#.###.#
#...J...#.#y....#.W.#...#.....#.#...#.#.#.#.....#.....#.#...#...........#.#.....#
#.#######.#########.#.#.#.#####.#.###.#.#.#####.#######.#.###.#.###.#####.#.#####
#.#...#...#.......#.#.#...#.....#.#.....#.......#.......#.#...#...#.#...#...#...#
#.#.#.#.#######.#V#.###.###.#####.#.#####.#########.#.#.#.#####.#.###.#Z#####.#.#
#.#v#.#.#.....#.#.#.#.#...#.#...#.#.....#.........#.#.#.#...#.#.#.....#.......#.#
#.###R#C#.###.#.#.#.#.#####.###.#.#####.#########.###.#.###.#.#################.#
#.#...#.#.#.#...#w#.#.....#...#...#...#.#.......#.....#...#...#.............#...#
#.#.###.#.#.#####.#.#####.###.#.###.#.#.#.#####.#########.#####.###########.#.#.#
#.#.#...#.#.........#...#.....#.#...#.#.#.A.#...........#...#...#..j......#.G.#.#
#.#.#.###.#.#######.###.#######.#.###.#####.#.#######.#####.#.###.#######.#####.#
#.#..r....#.#.....#.......#...#...#.#...#...#.#...#...#.....#.#...#.....#.......#
#.#######.###D###.#######.#.#.#####.###.#.###.#.#.#.###.#####.#.###.###.#########
#..z....#.#...#.#.....#t....#.#.......#.#...#.#i#.#...#.#.....#.#.....#.....#...#
#.#####.###.###.#####.#######.#######.#.#.#.###.#.#####.###O###.#####.#####.###.#
#.#...#...#.#.#.....#...#...#.......#.#.#.#.#...#.....#.#...#...#...#...#.......#
#.###.###.#.#.#.###.###.###.#.#####.#.#.###.#.#######.#.#.###.###.#.###.#######.#
#.....#.#.#.#.....#...#...#...#...#..f#.#...#.#.....#...#...#.....#...#...#...#.#
#####.#.#.#.#.#######.###.#####.#.#####.#.###.###.#.#######.#########.#.#.#.#.#.#
#...#.#.#...#...#.....#...#.....#.....#.#...#...#.#.#.......#...#.....#.#.#.#.#.#
#.#.#.#.#######.#.###.#.#.#.#########.#.#.#.###.#.#.#.#######.#.#.#######.#.#.#.#
#.#.#...#.....#.#.#...#.#.#.....#.#...#.#.#...#.#.#.#.#.....#.#...#.....#...#.#.#
#.#.###.###.#.###.#.###.#######.#.#.###.#.###.#.#.#.#.#.#####.#######.#.#####.#.#
#.#.........#.....#...#...........#.........#.....#...#...............#.......#.#
#######################################.@.#######################################
#.....#.........#.............#.#...................#.......#.......#.....#.....#
#.#.#.#.#######.#.###########.#.#.#.#####.###########.###.#.#.#####.#.###.#.###.#
#.#.#.#.#.....#.#.#.......#...#...#.#...#.......#...#.#.#.#.#.#...#.....#.#.#.#.#
#.#.#.#.#.###.#.#.###.#.###.###.###.#.#.#.#####.#.#.#.#.#.#.###.#.#######.#.#.#.#
#.#.#...#...#.#.#...#.#.#...#...#.#...#.#.#...#...#.....#.#.....#.......#.....#.#
###.#####.#.###.#.#.###.#.###.###.#####.#.#.#.###########.#############.#######.#
#...#.#...#.....#.#...#...#...#.......#.#...#.......#.....#.............#...#...#
#.###.#.#########.###.#.###.#########.#.###.#####.###.#########.#######.#.#.#.###
#.#.......#.....#...#.#.#.......#...#.#.#a..#...#.#...#.......#.#.....#d..#.#...#
#.#######.###.#.#.###.#.#######.#.#.#.#.#####.#.###.###.#####.#.#.###Y#####.#.#.#
#...#...#.....#.#.#...#.....I.#...#...#.#...#.#.....#...#.#...#.#.#...#...#.#.#.#
###.#.#.#######.###.#########.#########.#.#.#.#####.#.###.#.###.#.#.#####.#.###.#
#.#...#...#.........#.......#...........#.#.#.#.....#.#...#.#...#h#.#.....#.....#
#.#######.###########.#################.#.#.#.#######.#.#.#.#####.#.#.#########.#
#...#...#.....#...#.......#.......#.....#.#.#.....#...#.#.#.#.....#.#.#.......#.#
#.#.###.#####.#.#.#.#####.#.#.#####.#####.#.#####.#.###.#.#.#.#####.#.#.###.###.#
#.#.....#.#...#.#...#...#g#.#...#...#...#.#.#.....#.#...#.#.#.#...#.#.#...#.....#
#.#####.#.#.#########.#.#.#.###.#.###.#.#.#.#.###.#.#####.#.#.#.#.#.#.###.#######
#.#.....#.#...........#.#.#...#.#...#.#.#.#.#.#...#.....#.#.#.#.#.#.#...#...#...#
#.#.#####.#############.#.###.#.###.#.#.#.#.#.###.#####.#.#.#.#.###.#.#####.#.#.#
#.#...#.....#.#.....#...#...#.#o....#.#.#.#.#...#.#.....#.#.#c#...K.#.#.....#.#.#
#.###E#.###.#.#.###.#####.#.#.#########.#.#.###.#.#.#####.#.###.#####.#.#####.###
#.#...#...#.#.#...#...#...#...#.#.....#.#.#.....#.#.......#...#.....#.#.#...#...#
#.###.###.#.#.###.###.#.#######.#.#.#.#.#.#######.#######.#.#######.#.#.#.###.#.#
#...#.....#...#.#.#.#...#.....#...#.#...#.#.......#.....#.#...#.....#.#.#.....#.#
###.#.#######.#.#.#.#####.#.###.###.###.#.#.###########.#.###.#.#####.#.#######.#
#...#.#...#.....#.#..s#...#.....#...#.#.#.#.#.....#.....#.#.#...#.....#.....#...#
#.#.###.#.#####.#.###.#.#########.###.#.#.#.###.#.#.###.#.#.#####.#.#######.#.###
#.#.#...#.....#.#.#...#...#...#...#...#.#.#.....#...#...#...#...#.#.....#...#...#
#.###.#######.###.#.#.###.#.#.#.###.#.#.#.###########.#####.#.###.###M###.#####.#
#.#...#.....#.....#.#...#...#.#.#...#.#.#.#...........#.....#.....#.#...#.......#
#.#.#######.#######.#######.###.#.#.###.#.###########.#.#####.#####.###.#######.#
#.#.......#.#.....#...#.....#...#.#.#...#.......#...#.#.....#...#.....#.......#.#
#.#######.#.#.#.#.###.#.#####B###.###.###.#####.#.#.#######.###.#####.###.#####.#
#...#...#.#...#.#...#..l#...#...#...#.#.#.#...#.#.#.#.....#...#.#.....#...#...#.#
#.#.#.#.#.#.###.###.#.###.#.###.###.#P#.#.#.#Q#.#.#.#.###.###.#.#.###.#.###.#.#.#
#.#.#.#.#.#.#...#.#.#.#...#.....#...#...#...#.#.#.#.#...#...#.#...#...#...#.#...#
#.#.#.#.#.###.###.#.###.#########.#####.#####.#.#.#.###.###.#.#####.#####.#.#####
#.#...#.......#........u#.......X.......#.....#..n#.......#...#....e....#.......#
#################################################################################"""

maze, origin, keys, doors = parse_maze(mazestr)

print(len(maze), len(maze[0]))

print('maze under consideration')
for row in maze:
    print(''.join([c for c in row]))
    
print('\norigin', origin)
print('\nkey locations')
for key in keys:
    print(key, keys[key])
    
print('\ndoor locations')
for door in doors:
    print(door, doors[door])
    
print()
d, p = collect_keys(maze, origin, keys, doors)
print('\nbest:', d, p)

81 81
maze under consideration
#################################################################################
#m....#...#...#.....#...#...............#.#...#.....#...........................#
#.###.#.#.#.#.###.#.#.#.#####.#.#########.#.#.#.#.###.###.#.#############.#####.#
#...#...#...#...#.#b#.#.#...#.#........x#.#.#.#.#.....#...#.#.......#.....#...#.#
#.#.###########.#.#.#.#.#.#.###########.#.#.#.#.#######.#####.#####.#######.#.###
#.#.......#...#.U.#...#...#...........#.#.#.#...#.....#..........p#.........#...#
#.#######.#.#.#######################.#.#.#.#####.###.###########.#############.#
#...#...#.#.#.#...#.S...#...........#...#.#.#...#.#.#.....#....k#.....#.......#.#
###.#.#.#.#.#.#.#.#.###.#.###.#####L###.#.#.#.#.#.#.###N###.###.#######.#####.#.#
#...#.#...#.#.#.#...#.#.#...#.#.......#.#.#...#...#.....#...#.#.#.......#...#.#.#
#.###.#####.#.#.#####.#.###.#.#########.#.#########.#####.###.#.#.#######.#.#.#.#
#.#.#.#...#.#.#.#.T...#.....#.........#.#...#.....#...#.#...#.....#

5426 ['o', 'i', 'g', 'a', 'd', 'z', 'j', 'n', 'k', 'q', 'p', 'u', 'm', 'b', 'l', 's', 'y', 'e', 'h', 'c', 'x', 'r', 'v', 'w', 'f', 't']
new global best!
5408 ['o', 'i', 'g', 'a', 'd', 'z', 'j', 'n', 'k', 'q', 'p', 'u', 'm', 'b', 'l', 's', 'y', 'e', 'h', 'c', 'x', 'r', 'v', 'w', 't', 'f']
new global best!
5404 ['o', 'i', 'g', 'a', 'd', 'z', 'j', 'n', 'k', 'q', 'u', 'm', 'b', 'l', 's', 'y', 'e', 'h', 'c', 'p', 'x', 'r', 'v', 'w', 't', 'f']
new global best!
5126 ['o', 'i', 'g', 'a', 'd', 'z', 'j', 'n', 'k', 'p', 'q', 'm', 'e', 'u', 'b', 'l', 's', 'y', 'x', 'h', 'c', 'r', 'v', 'w', 'f', 't']
new global best!
5108 ['o', 'i', 'g', 'a', 'd', 'z', 'j', 'n', 'k', 'p', 'q', 'm', 'e', 'u', 'b', 'l', 's', 'y', 'x', 'h', 'c', 'r', 'v', 'w', 't', 'f']
new global best!
5066 ['o', 'i', 'g', 'a', 'd', 'z', 'j', 'n', 'k', 'p', 'q', 'm', 'u', 'b', 'l', 's', 'y', 'e', 'h', 'c', 'x', 'r', 'v', 'w', 'f', 't']
new global best!
5048 ['o', 'i', 'g', 'a', 'd', 'z', 'j', 'n', 'k', 'p', 'q', 'm', 'u', 'b', 'l', '